In [1223]:
## F1 practice and grid data from 2003 to 2024. 

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [1224]:
## Put all the possible years in a list.
url = 'https://pitwall.app/races'
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
ul = soup.find('ul', attrs = {'class':'options'})
possible_years = [x.text for x in ul.find_all('a')]

In [1225]:
## Define a function that converts minutes to seconds.
def convert(t):
    if ':' in t:
        sec_from_min = float(t.split(':')[0]) * 60
        sec = float(t.split(':')[1])
        return sec_from_min + sec
    else:
        return float(t)

In [1226]:
# test with race 8
F1 = pd.DataFrame(columns=['Season',
                               'Grand Prix',
                               'Driver',
                               'Constructor',
                               'FP1 Time','FP1 Gap','FP1 Laps',
                               'FP2 Time','FP2 Gap','FP2 Laps',
                               'FP3 Time','FP3 Gap','FP3 Laps'])

## You can slice possible_years
for year in possible_years:
    url = 'https://pitwall.app/races/archive/' + year
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    tbody = soup.find('tbody')
    possible_races = [x['href'] for x in tbody.find_all('a')]
#     possible_races = [x for x in possible_races 
#                       if x.split('races/')[1][5:].split('/')[0] != 'austrian-grand-prix' 
#                       and x.split('races/')[1][5:].split('/')[0] != 'belgian-grand-prix' 
#                       and x.split('races/')[1][5:].split('/')[0] != 'qatar-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'united-states-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'brazilian-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'azerbaijan-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'emilia-romagna-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'british-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'italian-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'russian-grand-prix'
#                       and x.split('races/')[1][5:].split('/')[0] != 'styrian-grand-prix']

    ## You can slice possible races
    for race in possible_races:
        url = 'https://pitwall.app/races/' + year + race[11:]
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        print('')
        print('working on', race)
        print('')
        
        ### Get the Free Practice data ###
        
        ## Get FP1
        FP1 = soup.find('div', attrs = {'id':'free_practice_1'})
        data_1 = []
        if FP1 != None:
            FP1_table = FP1.find('table', attrs = {'class':'data-table'})
            body = FP1_table.find('tbody')
            rows_1 = body.find_all('tr')
            data_1 = []
            for row in rows_1:
                row_data = []
                for cell in row.find_all('td')[1:]:
                    row_data.append(cell.text)
                ## Name
                name = row_data[0].split(' ')[2]
                row_data[0] = name
                ## Time
                if row_data[2] != '':
                    row_data[2] = round(convert(row_data[2]), 3)
                else:
                    row_data[2] = np.nan
                ## Gap
                if row_data[3] == '':
                    row_data[3] = 0 
                else:
                    if row_data[3] != '':
                        row_data[3] = float(convert(row_data[3].replace('+','')))
                    else:
                        row_data[3] = np.nan
                ## Lap
                if row_data[4] != '':
                    row_data[4] = float(row_data[4])
                else:
                    row_data[4] = np.nan
                data_1.append(row_data)
            print(data_1)
            print('')
        else:
            print('FP1 does not exists')
            print('')
        
        ## Get FP2
        FP2 = soup. find('div', attrs = {'id':'free_practice_2'}) 
        data_2 = []
        if FP2 != None:
            FP2_table = FP2.find('table', attrs = {'class':'data-table'})
            body = FP2_table.find('tbody')
            rows_2 = body.find_all('tr')
            for row in rows_2:
                row_data = []
                for cell in row.find_all('td')[1:]:
                    row_data.append(cell.text)
                ## Name
                name = row_data[0].split(' ')[2]
                row_data[0] = name
                ## Time
                if row_data[2] != '':
                    row_data[2] = round(convert(row_data[2]), 3)
                else:
                    row_data[2] = np.nan
                ## Gap
                if row_data[3] == '':
                    row_data[3] = 0 
                else:
                    if row_data[3] != '':
                        row_data[3] = float(convert(row_data[3].replace('+','')))
                    else:
                        row_data[3] = np.nan
                ## Lap
                if row_data[4] != '':
                    row_data[4] = float(row_data[4])
                else:
                    row_data[4] = np.nan
                data_2.append(row_data)
            print(data_2)    
            print('')
            ## Update data_1
            for row in data_1:
                if row[0] in [x[0] for x in data_2]:
                    index = [x[0] for x in data_2].index(row[0])
                    for i in data_2[index][2:]:
                        row.append(i)
                else:
                    row = [y for x in [row, [np.nan, np.nan, np.nan]] for y in x]
        else:
            print('FP2 does not exists')
            print('')
        
        ## Get FP3
        FP3 = soup.find('div', attrs = {'id':'free_practice_3'})
        data_3 = []
        if FP3 != None:
            FP3_table = FP3.find('table', attrs = {'class':'data-table'})
            body = FP3_table.find('tbody')
            rows_3 = body.find_all('tr')
            for row in rows_3:
                row_data = []
                for cell in row.find_all('td')[1:]:
                        row_data.append(cell.text)
                ## Name
                name = row_data[0].split(' ')[2]
                row_data[0] = name
                ## Time
                if row_data[2] != '':
                    row_data[2] = round(convert(row_data[2]), 3)
                else:
                    row_data[2] = np.nan
                ## Gap
                if row_data[3] == '':
                    row_data[3] = 0 
                else:
                    if row_data[3] != '':
                        row_data[3] = float(convert(row_data[3].replace('+','')))
                    else:
                        row_data[3] = np.nan
                ## Lap
                if row_data[4] != '':
                    row_data[4] = float(row_data[4])
                else:
                    row_data[4] = np.nan
                data_3.append(row_data)
            print(data_3)
            print('')
            ## Update data_1
            for row in data_1:
                if row[0] in [x[0] for x in data_3]:
                    index = [x[0] for x in data_3].index(row[0])
                    for i in data_3[index][2:]:
                        row.append(i)
                else:
                    row = [y for x in [row, [np.nan, np.nan, np.nan]] for y in x]
        else:
            print('FP3 does not exists')
            print('')
        
        ### Get the Starting grid ###
        grid = soup.find('div', attrs = {'id':'qualifying'})
        if grid != None:
            grid_table = grid.find('table', attrs = {'class':'data-table'})
            body = grid_table.find('tbody')
            rows_grid = body.find_all('tr')
            row_data = []
            for row in rows_grid:
                for cell in row.find_all('td')[1:2]:
                    name = cell.text
                    row_data.append(name)
            row_data = [x.split(' ')[2] for x in row_data]
            
            ## Update data_1
            for row in data_1:
                if row[0] in row_data:
                    index = row_data.index(row[0])
                    row.append(index + 1)
                else:
                    row.append(np.nan)
        else:
            print('Grid does not exist')
            print('')
        
        ### Update the data frame ###
        if (FP1 != None) and (FP2 != None) and (FP3 != None) and (grid != None):
            ## Make clean dataframes for the current race and year.
            L = [year, race[12:]]
            data_1 = [L + x for x in data_1]
            df = pd.DataFrame(data_1, columns=['Season',
                                   'Grand Prix',
                                   'Driver',
                                   'Constructor',
                                   'FP1 Time','FP1 Gap','FP1 Laps',
                                   'FP2 Time','FP2 Gap','FP2 Laps',
                                   'FP3 Time','FP3 Gap','FP3 Laps',
                                   'Grid Position'])
            F1 = pd.concat([F1, df])
            


working on /races/2023-bahrain-grand-prix

[['Pérez', 'Red Bull', 92.758, 0, 21.0], ['Alonso', 'Aston Martin', 93.196, 0.438, 22.0], ['Verstappen', 'Red Bull', 93.375, 0.617, 21.0], ['Norris', 'McLaren', 94.165, 1.407, 21.0], ['Leclerc', 'Ferrari', 94.257, 1.499, 17.0], ['Stroll', 'Aston Martin', 94.298, 1.54, 17.0], ['Magnussen', 'Haas', 94.402, 1.644, 20.0], ['Zhou', 'Alfa Romeo', 94.575, 1.817, 18.0], ['Bottas', 'Alfa Romeo', 94.689, 1.931, 20.0], ['Hamilton', 'Mercedes', 94.917, 2.159, 13.0], ['Russell', 'Mercedes', 94.966, 2.208, 20.0], ['Piastri', 'McLaren', 94.997, 2.239, 24.0], ['Tsunoda', 'AlphaTauri', 95.015, 2.257, 20.0], ['Hülkenberg', 'Haas', 95.043, 2.285, 22.0], ['Ocon', 'Alpine', 95.105, 2.347, 20.0], ['de', 'AlphaTauri', 95.402, 2.644, 25.0], ['Gasly', 'Alpine', 95.455, 2.697, 22.0], ['Sargeant', 'Williams', 95.749, 2.991, 24.0], ['Albon', 'Williams', 96.018, 3.26, 15.0], ['Sainz', 'Ferrari', 96.072, 3.314, 21.0]]

[['Alonso', 'Aston Martin', 90.907, 0, 25.0], ['Verst

/var/folders/yd/kp6bpgzj2x5211sc99wjp9z80000gn/T/ipykernel_23089/2017073521.py:210: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  F1 = pd.concat([F1, df])



working on /races/2023-saudi-arabian-grand-prix

[['Verstappen', 'Red Bull', 89.617, 0, 18.0], ['Pérez', 'Red Bull', 90.1, 0.483, 20.0], ['Alonso', 'Aston Martin', 90.315, 0.698, 22.0], ['Stroll', 'Aston Martin', 90.577, 0.96, 20.0], ['Russell', 'Mercedes', 90.771, 1.154, 26.0], ['Hamilton', 'Mercedes', 90.787, 1.17, 26.0], ['Sainz', 'Ferrari', 90.924, 1.307, 23.0], ['Gasly', 'Alpine', 90.949, 1.332, 24.0], ['Albon', 'Williams', 91.03, 1.413, 23.0], ['Tsunoda', 'AlphaTauri', 91.11, 1.493, 25.0], ['Leclerc', 'Ferrari', 91.118, 1.501, 26.0], ['Ocon', 'Alpine', 91.181, 1.564, 22.0], ['de', 'AlphaTauri', 91.45, 1.833, 29.0], ['Piastri', 'McLaren', 91.491, 1.874, 24.0], ['Hülkenberg', 'Haas', 91.552, 1.935, 22.0], ['Magnussen', 'Haas', 91.566, 1.949, 23.0], ['Sargeant', 'Williams', 91.922, 2.305, 26.0], ['Bottas', 'Alfa Romeo', 91.97, 2.353, 25.0], ['Zhou', 'Alfa Romeo', 91.986, 2.369, 22.0], ['Norris', 'McLaren', 92.149, 2.532, 21.0]]

[['Verstappen', 'Red Bull', 89.603, 0, 29.0], ['Alons


working on /races/2023-monaco-grand-prix

[['Sainz', 'Ferrari', 73.372, 0, 26.0], ['Alonso', 'Aston Martin', 73.71, 0.338, 28.0], ['Hamilton', 'Mercedes', 74.035, 0.663, 27.0], ['Pérez', 'Red Bull', 74.038, 0.666, 27.0], ['Leclerc', 'Ferrari', 74.093, 0.721, 24.0], ['Verstappen', 'Red Bull', 74.244, 0.872, 26.0], ['Norris', 'McLaren', 74.467, 1.095, 25.0], ['Ocon', 'Alpine', 74.585, 1.213, 29.0], ['Stroll', 'Aston Martin', 74.653, 1.281, 28.0], ['Albon', 'Williams', 74.666, 1.294, 32.0], ['Bottas', 'Alfa Romeo', 74.718, 1.346, 29.0], ['Magnussen', 'Haas', 74.725, 1.353, 29.0], ['Tsunoda', 'AlphaTauri', 74.82, 1.448, 29.0], ['Gasly', 'Alpine', 74.866, 1.494, 30.0], ['Russell', 'Mercedes', 75.066, 1.694, 24.0], ['de', 'AlphaTauri', 75.083, 1.711, 33.0], ['Piastri', 'McLaren', 75.192, 1.82, 32.0], ['Sargeant', 'Williams', 75.557, 2.185, 30.0], ['Zhou', 'Alfa Romeo', 75.684, 2.312, 25.0], ['Hülkenberg', 'Haas', 75.785, 2.413, 17.0]]

[['Verstappen', 'Red Bull', 72.462, 0, 30.0], ['Leclerc


working on /races/2023-british-grand-prix

[['Verstappen', 'Red Bull', 88.6, 0, 26.0], ['Pérez', 'Red Bull', 89.048, 0.448, 22.0], ['Albon', 'Williams', 89.089, 0.489, 22.0], ['Alonso', 'Aston Martin', 89.268, 0.668, 21.0], ['Leclerc', 'Ferrari', 89.28, 0.68, 25.0], ['Ocon', 'Alpine', 89.319, 0.719, 23.0], ['Sainz', 'Ferrari', 89.357, 0.757, 25.0], ['Norris', 'McLaren', 89.441, 0.841, 25.0], ['Stroll', 'Aston Martin', 89.471, 0.871, 26.0], ['Piastri', 'McLaren', 89.658, 1.058, 16.0], ['de', 'AlphaTauri', 89.691, 1.091, 26.0], ['Hamilton', 'Mercedes', 89.768, 1.168, 23.0], ['Gasly', 'Alpine', 89.828, 1.228, 24.0], ['Russell', 'Mercedes', 89.874, 1.274, 28.0], ['Bottas', 'Alfa Romeo', 90.09, 1.49, 22.0], ['Tsunoda', 'AlphaTauri', 90.092, 1.492, 28.0], ['Sargeant', 'Williams', 90.124, 1.524, 24.0], ['Zhou', 'Alfa Romeo', 90.321, 1.721, 20.0], ['Magnussen', 'Haas', 90.385, 1.785, 21.0], ['Hülkenberg', 'Haas', 90.591, 1.991, 22.0]]

[['Verstappen', 'Red Bull', 88.078, 0, 27.0], ['Sainz', '

...


working on /races/1980-united-states-grand-prix-west

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-belgian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-monaco-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-french-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-british-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-german-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-austrian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-dutch-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-italian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1980-canadi


working on /races/1975-brazilian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-south-african-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-spanish-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-monaco-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-belgian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-swedish-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-dutch-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-french-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-british-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1975-german-gr


working on /races/1970-united-states-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1970-mexican-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-south-african-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-spanish-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-monaco-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-dutch-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-french-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-british-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-german-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1969-italia


working on /races/1962-belgian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1962-french-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1962-british-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1962-german-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1962-italian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1962-united-states-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1962-south-african-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1961-monaco-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1961-dutch-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1961-belgia


working on /races/1953-argentine-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-indianapolis-500

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-dutch-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-belgian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-french-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-british-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-german-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-swiss-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1953-italian-grand-prix

FP1 does not exists

FP2 does not exists

FP3 does not exists


working on /races/1952-swiss-grand-prix

F

In [1296]:
## Finally, we drop the NaN values.
F1 = F1.dropna()

In [1298]:
F1.sample(5)

,Season,Grand Prix,Driver,Constructor,FP1 Time,FP1 Gap,FP1 Laps,FP2 Time,FP2 Gap,FP2 Laps,FP3 Time,FP3 Gap,FP3 Laps,Grid Position
5,2015,japanese-grand-prix,Verstappen,Toro Rosso,110.940,1.506,10.0,110.542,2.265,5.0,95.160,1.165,26.0,15.0
18,2010,european-grand-prix,Kovalainen,Lotus Racing,104.491,3.316,21.0,102.467,3.184,31.0,101.303,3.251,19.0,20.0
7,2019,german-grand-prix,Grosjean,Haas,75.074,1.061,30.0,74.179,0.730,33.0,73.378,0.998,15.0,6.0
8,2021,french-grand-prix,Norris,McLaren,94.707,1.259,21.0,93.822,0.950,24.0,92.336,1.036,14.0,8.0
9,2018,hungarian-grand-prix,Magnussen,Haas,79.187,1.574,29.0,78.880,2.046,35.0,78.230,2.060,17.0,9.0


In [1301]:
# filename = 'F1Data.csv'

In [1302]:
# F1.to_csv(filename, sep=',', index=False, encoding='utf-8')